# Table Cleaning
---
This notebook was used during the table cleaning process. This notebook does not represent the entire cleaning process. Many tables had to be looked at and cleaned manually. Below is a basic outline of the issue encountered and the subsequent solution.

---
### 1. Some tables were only in excel format, which is difficult/slow to read in Python. Also, Excel sheets were in disparate formats (.xlsx, .xls).
- Converted excel sheets to .txt when no .txt was present
- Converted occasional .csv to .txt
- Converted many excel sheets into a standard .xls format; not all were converted because I realized I wasn't going to be using the excel sheets directly
### 2. Some tables had 1, 2, or 3 column headers.
- I manually opened the .txt and deleted unneccessary headers (headers describing the tower or headers describing units)
    - How many headers depended on the file, so I found it faster to do this by hand
### 3. Many months (not all) were split into 2 to 3 subfiles that were suffixed with a letter.
- E.g. `TOWA_15M_201701A.txt` and `TOWA_15M_201701B.txt` sould be one january file: `TOWA_15M_201701.txt`
- I used Python to merge subfiles into one monthy file; pandas row-wise concatenatination by column name
    - Column names were not consistent among subfiles. So some data were split into multiple columns.
### 4. Column names were inconsistent
- 15m could indicate the variable was collected at 15-meters or 15-minutes, which was confusing at first
    - This was later clarified when learning at which heights data were collected at each tower
- Some column names erroneously indicated 60-minute data with 001h, so I switched them to 15m
- Sometimes parenthesis were present and had to be removed
- Hanging whitespace was removed and spaces were replaced with an underscore
- Some column headers were prefixed with a number; these were identified and removed
### 5. The meaning of variable names were unclear. Which variables were the same but had different names took significant time to understand and fix.
- I manually aligned variable names (all column names meaning one thing were put into one row together)
    - This was done manually and iteratively because I sometimes missed variables or aligned them incorrectly
        - see `{towername}_var_names_{year}_unaligned.csv` and `{towername}_var_names_{year}_aligned.csv` in `header_cleaning` directories
- After alignment, I automatically assigned one common name (first non-nan name) for variable names that meant the same thing but were typed differently
    - E.g. WBulbC_010m, WBulbC, WetBulbC_010m, WetBulbC_001h for ONE Tower were renamed to "WetBulbC_010m"
        - Eventually, variable names were iteratively standardized across all towers (usually manually)
        - Standardized naming convention: VariableNameUnit_Heightm (e.g. WetBulbC_010m or BarPresMb_010m or PkWSpdMph_010m); **current naming convention isn't acceptable. It should be changed.**
### 6. A standardized name for Tower A may not be the standardized name for Tower B
- After standardizing headers in the monthly tables for each tower, I noticed that the towers did not use the same naming convention either; so I had to re-do step five but for all the towers.
    - See: `ALLT_var_names_{YEAR}_standard.csv'`
- I repeated this process for each year, because year-to-year the variable names were not the same
### 7. There were still issues after merging (sometimes errors on my part)
- After the monthly tables were merged into multi-annual files like `TOWA_2017-2022.csv`, I noticed that some columns were not complete
    - They were not complete because of lingering issues with variable names; I manually combined columns that were accidentally split apart.
- When copying/pasting between excel sheets, some calculations were done in the sheet; so those values were messed up and sometimes I didn't notice until later
- Lingering -999 values (sometimes -999.99 or -1000) were sometimes used to calculate a variable, which I later identified and removed
- Sometimes sensors outputted near identical values for many hours/days, which is a sensor error; this shows up as real data, but had to be identified and removed after visually analyzing graphs over time of each variable; any strange looking patterns were noted and visually assessed; many were corrected, but likely not all

---

## Import modules

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import pprint

## Set parameters
I ran this file for each year; so the YEAR variable was changed often

In [4]:
# edit parameters
YEAR = 2017
DATASET = 'ornl_15m'
DATA_DIR = '/home/6ru/Desktop/nsrd/data'

# optionally edit when needed:
DIR = f'{DATA_DIR}/{DATASET}/{YEAR}'
DATA_FILES = sorted(glob.glob(f'{DIR}/TOW*/*.txt'))
HEADER_DIR = f'{DATA_DIR}/{DATASET}/{YEAR}/header_cleaning'
CLEAN_DIR = f'{DATA_DIR}/{DATASET}/{YEAR}/clean_files'
IMG_DIR = f'{DATA_DIR}/{DATASET}/images/{DATASET}_{YEAR}'

## Loop through towers and group months

In [5]:
# create list of tower names
tower_names = []
for data in DATA_FILES:
    file_name = data.split('/')[-1]
    tower_name = file_name.split('_')[0]
    tower_names.append(tower_name)
unique_tower_names = sorted(list(set(tower_names)))
unique_tower_names

['TOWA', 'TOWB', 'TOWD', 'TOWF', 'TOWS', 'TOWY']

In [8]:
# create list of dates that exists for each tower; towers may be missing months
dates_per_tower = []
for tname in unique_tower_names:
    
    tower_dates = []
    for dataf in DATA_FILES:

        file_name = dataf.split('/')[-1] #towa_15m_dateA.txt
        file_name_noext = file_name.split('.')[0] #towa_15m_dateA
        tower_name = file_name.split('_')[0] #towa
        
        if tname == tower_name:
            # if end of name is a letter
            if not file_name_noext[-1].isdigit():
                date = file_name_noext.split('_')[-1]
                date = date[:-1] # drop letter
            else:
                date = file_name_noext.split('_')[-1]
            tower_dates.append(date)
        else:
            continue
    tower_dates = sorted(list(set(tower_dates)))
    dates_per_tower.append(tower_dates)

## If months are split into sub-months, combine

In [11]:
towers_month_groups = []
for uname, tdates in zip(unique_tower_names, dates_per_tower):
    
    grouped_by_date = []
    
    for tdate in tdates:
        
        files_per_date = []
        
        for dfile, tname in zip(DATA_FILES, tower_names):
            
            file_name = dfile.split('/')[-1] #towa_15m_dateA.txt
            file_name_noext = file_name.split('.')[0] #towa_15m_dateA

            if tname == uname:
                # create date
                if not file_name_noext[-1].isdigit():
                    date = file_name_noext.split('_')[-1]
                    date = date[:-1] # drop letter
                else:
                    date = file_name_noext.split('_')[-1]
                    
                # if dates are the same
                if tdate == date:
                    files_per_date.append(dfile)
                    
        grouped_by_date.append(files_per_date)
    towers_month_groups.append(grouped_by_date)

In [14]:
# Tower A [0] first month [0] files
towers_month_groups[0][0]

['/home/6ru/Desktop/nsrd/data/ornl_15m/2017/TOWA_15M_2017/TOWA_15M_201701A.txt',
 '/home/6ru/Desktop/nsrd/data/ornl_15m/2017/TOWA_15M_2017/TOWA_15M_201701B.txt']

## Read data

In [15]:
# function to read data
def read_and_clean(path, tower_name):
    print(path)
    df = pd.read_csv(path, index_col=0, header=0)
    df.index = pd.to_datetime(df.index, format='mixed')
    df['Tower'] = tower_name
    return df

In [16]:
# read files for each tower
towers_month_dfs = []
for tower, utname in zip(towers_month_groups, unique_tower_names):
    month_dfs = []
    print(utname)
    for month in tower:
        if len(month) > 1:
            month_files = []
            for submonth in month:
                df = read_and_clean(submonth, utname)
                month_files.append(df)
            month_df = pd.concat(month_files)
            month_dfs.append(month_df)
        else:
            month_df = read_and_clean(month[0], utname)
            month_dfs.append(month_df)
    towers_month_dfs.append(month_dfs)

TOWA
/home/6ru/Desktop/nsrd/data/ornl_15m/2017/TOWA_15M_2017/TOWA_15M_201701A.txt
/home/6ru/Desktop/nsrd/data/ornl_15m/2017/TOWA_15M_2017/TOWA_15M_201701B.txt
/home/6ru/Desktop/nsrd/data/ornl_15m/2017/TOWA_15M_2017/TOWA_15M_201702A.txt
/home/6ru/Desktop/nsrd/data/ornl_15m/2017/TOWA_15M_2017/TOWA_15M_201702B.txt
/home/6ru/Desktop/nsrd/data/ornl_15m/2017/TOWA_15M_2017/TOWA_15M_201703A.txt
/home/6ru/Desktop/nsrd/data/ornl_15m/2017/TOWA_15M_2017/TOWA_15M_201703B.txt
/home/6ru/Desktop/nsrd/data/ornl_15m/2017/TOWA_15M_2017/TOWA_15M_201704A.txt
/home/6ru/Desktop/nsrd/data/ornl_15m/2017/TOWA_15M_2017/TOWA_15M_201704B.txt
/home/6ru/Desktop/nsrd/data/ornl_15m/2017/TOWA_15M_2017/TOWA_15M_201705A.txt
/home/6ru/Desktop/nsrd/data/ornl_15m/2017/TOWA_15M_2017/TOWA_15M_201705B.txt
/home/6ru/Desktop/nsrd/data/ornl_15m/2017/TOWA_15M_2017/TOWA_15M_201706A.txt
/home/6ru/Desktop/nsrd/data/ornl_15m/2017/TOWA_15M_2017/TOWA_15M_201706B.txt
/home/6ru/Desktop/nsrd/data/ornl_15m/2017/TOWA_15M_2017/TOWA_15M_201707

In [27]:
# column names that haven't been cleaned; tower A, month 0
towers_month_dfs[0][0].head(5)

,TempC_015m_015m,TempC_030m_015m,TempF_015m_015m,TempF_030m_015m,VT_Grad_015m,DewPtC_015m_015m,DewPtF_015m_015m,RH_015m_015m,AbsHum_015m_015m,MixRatio_015m_015m,...,VSSpd_030m_015m,VSSpdms_015m_015m,VSSpdms_030m_015m,PkWsQ_015m_015m,PkWsQ_030m_015m,SigPhi_015m_015m,SigPhi_030m_015m,SonicTemp_015m_015m,SonicTemp_030m_015m,Tower
2017-01-01 00:00:00,3.6,3.6,38.5,38.5,0.0,3.1,37.6,97.4,6.0,0.004857,...,-0.100000,0.02,-0.030000,5.9,10.2,18.9,12.6,4.7,4.0,TOWA
2017-01-01 00:15:00,3.6,3.6,38.5,38.5,0.0,3.1,37.6,97.3,6.0,0.004857,...,-0.100000,0.02,-0.030000,6.7,7.7,14.3,9.8,4.7,4.1,TOWA
2017-01-01 00:30:00,3.6,3.7,38.6,38.6,0.0,3.2,37.7,97.4,6.0,0.004889,...,0.000064,-0.09,0.000029,6.0,6.8,13.6,11.0,4.8,4.1,TOWA
2017-01-01 00:45:00,3.7,3.7,38.7,38.7,0.0,3.2,37.8,97.4,6.0,0.004919,...,-0.100000,-0.10,-0.040000,5.1,6.8,14.4,8.3,4.8,4.2,TOWA
2017-01-01 01:00:00,3.8,3.8,38.8,38.8,0.0,3.3,37.9,97.3,6.0,0.004915,...,0.100000,-0.01,0.020000,4.2,6.3,14.9,10.8,4.9,4.3,TOWA


In [18]:
# how many towers
len(towers_month_dfs)

6

In [20]:
# how many months of data in tower A
len(towers_month_dfs[0])

12

In [21]:
# function for general column name cleanup
def clean_column_names(column_names):
    
    new_names = []

    # loop through each col name
    for name in column_names:
        # remove anything in ()
        if ')' in name:
            name = name.split(')')[1]
        # remove leading/trailing space
        name = name.strip()
        # remove number at beginning
        if name[:1].isdigit():
            before, sep, after = name.partition('_')
            name = after
        # fill blank space with _
        name = name.replace(' ', '_')
        # replace 001h with 015m
        if '001h' in name:
            name = name.replace('001h', '015m')
        new_names.append(name)
        
    return new_names

In [22]:
# loop through towers and apply col extraction and general cleaning
towers_month_dfs_clean = []
for tower, utname in zip(towers_month_dfs, unique_tower_names):

    print(utname)
    tower_cleaned = []

    for month_df in tower:

        col_names = month_df.columns.to_list()
        clean_col_names = clean_column_names(col_names)
    
        # append
        tower_cleaned.append(clean_col_names)

    towers_month_dfs_clean.append(tower_cleaned)

TOWA
TOWB
TOWD
TOWF
TOWS
TOWY


In [23]:
# replace old column names with new ones for each df
for tower, clean_tower in zip(towers_month_dfs, towers_month_dfs_clean):
    for month_df, clean_month_list in zip(tower, clean_tower):
        month_df.columns = clean_month_list

In [26]:
# "cleaner" column names
towers_month_dfs[0][0].head(5)

,TempC_015m_015m,TempC_030m_015m,TempF_015m_015m,TempF_030m_015m,VT_Grad_015m,DewPtC_015m_015m,DewPtF_015m_015m,RH_015m_015m,AbsHum_015m_015m,MixRatio_015m_015m,...,VSSpd_030m_015m,VSSpdms_015m_015m,VSSpdms_030m_015m,PkWsQ_015m_015m,PkWsQ_030m_015m,SigPhi_015m_015m,SigPhi_030m_015m,SonicTemp_015m_015m,SonicTemp_030m_015m,Tower
2017-01-01 00:00:00,3.6,3.6,38.5,38.5,0.0,3.1,37.6,97.4,6.0,0.004857,...,-0.100000,0.02,-0.030000,5.9,10.2,18.9,12.6,4.7,4.0,TOWA
2017-01-01 00:15:00,3.6,3.6,38.5,38.5,0.0,3.1,37.6,97.3,6.0,0.004857,...,-0.100000,0.02,-0.030000,6.7,7.7,14.3,9.8,4.7,4.1,TOWA
2017-01-01 00:30:00,3.6,3.7,38.6,38.6,0.0,3.2,37.7,97.4,6.0,0.004889,...,0.000064,-0.09,0.000029,6.0,6.8,13.6,11.0,4.8,4.1,TOWA
2017-01-01 00:45:00,3.7,3.7,38.7,38.7,0.0,3.2,37.8,97.4,6.0,0.004919,...,-0.100000,-0.10,-0.040000,5.1,6.8,14.4,8.3,4.8,4.2,TOWA
2017-01-01 01:00:00,3.8,3.8,38.8,38.8,0.0,3.3,37.9,97.3,6.0,0.004915,...,0.100000,-0.01,0.020000,4.2,6.3,14.9,10.8,4.9,4.3,TOWA


## Manual Cleanup Stage (variable alignment)
Align column names in the csv files and create a tower_#_headers_year.csv file for each tower.

In [34]:
# look at cleaned columns and consolidate in excel
for tower, utname in zip(towers_month_dfs, unique_tower_names):
    cols = []
    utname_list = []
    for month_df in tower:
        df_cols = sorted(list(month_df.columns))
        cols.append(df_cols)
        utname_list.append(utname)
    col_df = pd.DataFrame(cols, utname_list)
    col_df = col_df.T
    col_df.to_csv(f'{HEADER_DIR}/{utname}_var_names_{YEAR}_unaligned.csv', index=False)

## Column header standardization (pt1 per tower)
Manually align header names and name the files from _unaligned to _aligned

In [32]:
# get aligned headers for each tower/year
header_paths = glob.glob(f'{HEADER_DIR}/*_var_names_{YEAR}_aligned.csv')
header_files = []
for unique_name in unique_tower_names:
    for header_path in header_paths:
        header_tower = header_path.split('/')[-1]
        header_tower = header_tower.split('_')[0]
        if header_tower == unique_name:
            header_files.append(header_path)

In [33]:
# read header files and sort names
header_dfs = []
for header_file in header_files:
    df = pd.read_csv(header_file)
    cols_in_order = sorted(list(df.columns))
    df1 = df[cols_in_order]
    header_dfs.append(df1)

In [34]:
# example of aligned header file
header_dfs[3].head(5)

,TOWF,TOWF.1,TOWF.2
0,NaN,AbsHum_010m_015m,AbsHum_010m_015m
1,NaN,BarPres_In_015m,BarPres_In_015m
2,NaN,BarPres_Mb_015m,BarPres_Mb_015m
3,NaN,DewPtC_010m_015m,DewPtC_010m_015m
4,NaN,DewPtF_010m_015m,DewPtF_010m_015m


In [35]:
# function to remove any nans from list
def remove_nan(row):
    new_list = [x for x in row if pd.notnull(x)]
    return new_list

In [40]:
# create lists of unique names for each row (var)
uniques = []
for df in header_dfs:
    u = df.apply(lambda x: list(set(x)), axis=1)
    uniques.append(u)

In [46]:
# number of variable names for a tower
len(uniques[0])

44

In [44]:
# show unique variable names
uniques[0][:5]

0    [AbsHum_015m_015m]
1     [BarPres_In_015m]
2     [BarPres_Mb_015m]
3    [DewPtC_015m_015m]
4    [DewPtF_015m_015m]
dtype: object

In [51]:
# gather new column names
new_tower_month_cols = []
i = 0
for tower, unames_list in zip(towers_month_dfs, uniques):
    
    # print('STARTING TOWER HERE')
    print(f'Tower #{i}')
    month_cols = []
    for month in tower:
        
        # print('STARTING MONTH HERE')
        orig_cols = month.columns
        new_cols = []
        
        for ocol in orig_cols: #loop thru 44 col names
            
            # print('Original column name: ', ocol)
            for lst in unames_list: # loop through n num lists of names
                nonnan_lst = remove_nan(lst)
                new_name = nonnan_lst[0]
                if ocol in lst:
                    # print('Matched! New name is: ', new_name)
                    new_cols.append(new_name)
                    break # stop looking for match
                else:
                    # print(f'Column {ocol} not found in {lst}')
                    continue

        # check for issues
        if len(orig_cols) != len(new_cols):
            print('Issue found. Mismatch in number of in-out column names:')
            print(len(orig_cols), len(new_cols))
            
        month_cols.append(new_cols)
        
    new_tower_month_cols.append(month_cols)
    i += 1

Tower #0
Tower #1
Tower #2
Tower #3
Tower #4
Tower #5
Issue found. Mismatch in number of in-out column names:
34 18


In [52]:
# replace old col names with new col names
i = 0
for tower, tcols in zip(towers_month_dfs, new_tower_month_cols):
    for month_df, tcol in zip(tower, tcols):
        try:
            month_df.columns = tcol
        except Exception as e:
            print(f'Issue with Tower #{i}. Unable to rename variables.')
    i += 1

Issue with Tower #5. Unable to rename variables.


In [54]:
towers_month_dfs[0][0].head(5)

,TempC_015m_015m,TempC_030m_015m,TempF_015m_015m,TempF_030m_015m,VT_Grad_015m,DewPtC_015m_015m,DewPtF_015m_015m,RH_015m_015m,AbsHum_015m_015m,MixRatio_015m_015m,...,VSSpd_030m_015m,VSSpdms_015m_015m,VSSpdms_030m_015m,PkWsQ_015m_015m,PkWsQ_030m_015m,SigPhi_015m_015m,SigPhi_030m_015m,SonicTemp_015m_015m,SonicTemp_030m_015m,Tower
2017-01-01 00:00:00,3.6,3.6,38.5,38.5,0.0,3.1,37.6,97.4,6.0,0.004857,...,-0.100000,0.02,-0.030000,5.9,10.2,18.9,12.6,4.7,4.0,TOWA
2017-01-01 00:15:00,3.6,3.6,38.5,38.5,0.0,3.1,37.6,97.3,6.0,0.004857,...,-0.100000,0.02,-0.030000,6.7,7.7,14.3,9.8,4.7,4.1,TOWA
2017-01-01 00:30:00,3.6,3.7,38.6,38.6,0.0,3.2,37.7,97.4,6.0,0.004889,...,0.000064,-0.09,0.000029,6.0,6.8,13.6,11.0,4.8,4.1,TOWA
2017-01-01 00:45:00,3.7,3.7,38.7,38.7,0.0,3.2,37.8,97.4,6.0,0.004919,...,-0.100000,-0.10,-0.040000,5.1,6.8,14.4,8.3,4.8,4.2,TOWA
2017-01-01 01:00:00,3.8,3.8,38.8,38.8,0.0,3.3,37.9,97.3,6.0,0.004915,...,0.100000,-0.01,0.020000,4.2,6.3,14.9,10.8,4.9,4.3,TOWA


## Merge monthly tables into annual table

In [55]:
# combine months into year
tower_year_dfs = []
for month_lst in towers_month_dfs:
    print('TOWER')
    for month_df in month_lst:
        columns = list(month_df.columns)
        duplicated = month_df.columns.duplicated()
        new_columns = []
        for col, dup in zip(columns, duplicated):
            if dup == True:
                new_columns.append(col+'_a')
            else:
                new_columns.append(col)
        month_df.columns = new_columns
    year = pd.concat(month_lst, axis=0)
    year.drop(columns=['Tower'], inplace=True)
    year = year.astype('float')
    tower_year_dfs.append(year)

TOWER
TOWER
TOWER
TOWER
TOWER
TOWER


In [56]:
tower_year_dfs[2].head(5)

,TempC_002m_015m,TempC_015m_015m,TempC_035m_015m,TempC_060m_015m,TempF_002m_015m,TempF_015m_015m,TempF_035m_015m,TempF_060m_015m,VTGrad_0215m_015m,VTGrad_0235m_015m,...,VSSpdms_060m_015m,PkWsQ_015m_015m,PkWsQ_035m_015m,PkWsQ_060m_015m,SigPhi_015m_015m,SigPhi_060m_015m,SonicTemp_015m_015m,SonicTemp_060m_015m,Precip_mm_015m,Precip_Sum_015m
2017-01-01 00:00:00,3.9,3.9,3.8,3.7,39.0,39.0,38.9,38.7,0.00,-0.08,...,-0.07,5.5,7.3,7.8,12.6,9.4,4.2,3.8,NaN,NaN
2017-01-01 00:15:00,3.9,3.9,3.8,3.7,39.0,39.0,38.8,38.6,-0.03,-0.11,...,-0.01,6.2,7.1,8.7,14.2,10.6,4.2,3.8,NaN,NaN
2017-01-01 00:30:00,3.9,3.9,3.9,3.8,39.1,39.1,39.0,38.9,-0.01,-0.06,...,0.01,3.7,6.0,7.1,14.5,10.4,4.3,4.0,NaN,NaN
2017-01-01 00:45:00,4.0,4.0,3.9,3.9,39.2,39.2,39.1,39.0,0.00,-0.06,...,0.05,3.2,5.3,6.4,13.8,10.1,4.3,4.0,NaN,NaN
2017-01-01 01:00:00,4.1,4.1,4.0,3.9,39.3,39.4,39.2,39.0,0.02,-0.07,...,-0.13,3.4,7.1,10.3,14.9,10.9,4.4,4.0,NaN,NaN


## Standardize cleaned names (pt2 all towers)
Do this manually in excel and export as *standard.csv

In [57]:
# export cleaned column names for standardization
headers = []
for year_df in tower_year_dfs:
    clean_headers = list(year_df.columns)
    headers.append(clean_headers)
headers_df = pd.DataFrame(headers, columns=None, index=None)
headers_df = headers_df.T
headers_df.to_csv(f'{HEADER_DIR}/ALLT_var_names_{YEAR}.csv')

## After creating ALLT_var_names_year_standard.csv:

In [48]:
# read standardized columns
stand_cols = pd.read_csv(f'{HEADER_DIR}/ALLT_var_names_{YEAR}_standard.csv', index_col=0, header=0)
num_towers = stand_cols.shape[1]
final_cols = []
for n in range(num_towers):
    cols = stand_cols[str(n)].tolist()
    new_list = [x for x in cols if pd.notnull(x)]
    final_cols.append(new_list)

In [49]:
# export unique column names
allcols_2d = stand_cols.values.tolist()
allcols_1d = [j for sub in allcols_2d for j in sub]
ac_1d_norepeats = [str(x).split('_')[0] for x in allcols_1d]
uvars = pd.DataFrame(sorted(list(set(ac_1d_norepeats))), 
                     columns=['Unique Variables'])
uvars.to_csv(f'{HEADER_DIR}/ALLT_unique_var_names_{YEAR}.csv', index=False)